<a href="https://colab.research.google.com/github/suhani121/Forest_fire/blob/main/Xception_X_mobilenetipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir = '/content/drive/MyDrive/forest fire'
labels = {'fire': 0, 'non fire': 1}  # Map categories to numeric labels

# Initialize lists to store image data and labels
image_data = []
image_labels = []

In [ ]:
for category in labels.keys():
    folder_path = os.path.join(data_dir, category)  # Path to category folder
    label = labels[category]  # Numeric label for the category

    for img_name in os.listdir(folder_path):  # Loop through images in the folder
        img_path = os.path.join(folder_path, img_name)
        try:
            # Skip non-image files
            if img_name.endswith('.jpg') or img_name.endswith('.png'):
                # Load and preprocess the image
                img = Image.open(img_path).convert('RGB')  # Ensure 3 channels
                img = img.resize((128, 128))  # Resize all images to 128x128
                image_data.append(np.array(img))  # Add image data
                image_labels.append(label)  # Add corresponding label
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

# Convert to NumPy arrays
X = np.array(image_data, dtype='float32') / 255.0  # Normalize image data to [0, 1]
y = np.array(image_labels)  # Convert labels to a NumPy array

# Verify shapes
print(f"X shape: {X.shape}")  # Should be (num_images, 128, 128, 3)
print(f"y shape: {y.shape}")  # Should be (num_images,)

X shape: (3085, 128, 128, 3)
y shape: (3085,)


In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder


from sklearn.model_selection import train_test_split

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify shapes
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")
print (y_train)


X_train shape: (2468, 128, 128, 3)
X_test shape: (617, 128, 128, 3)
y_train shape: (2468,)
y_test shape: (617,)
[1 1 0 ... 0 0 0]


In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
data_dir = '/content/drive/MyDrive/forest fire'
labels_map = {'fire': 0, 'non fire': 1}

image_data = []
image_labels = []

for category in labels_map.keys():
    folder_path = os.path.join(data_dir, category)
    label = labels_map[category]

    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            if img_name.lower().endswith(('.jpg', '.png')):
                img = Image.open(img_path).convert('RGB')
                img = img.resize((224, 224))  # Resize to match Xception input
                image_data.append(np.array(img))
                image_labels.append(label)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")

# Convert to arrays
X = np.array(image_data, dtype='float32') / 255.0
y = np.array(image_labels)

# split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Compute class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))

# Model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train
history = model.fit(X_train, y_train,
                    validation_split=0.1,
                    epochs=75,
                    batch_size=32,
                    class_weight=class_weights,
                    callbacks=[early_stop])

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc * 100:.2f}%")

# Save
model.save('xception_fire_detection_manual.h5')


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 119s 911ms/step - accuracy: 0.9279 - loss: 0.1593 - val_accuracy: 0.9555 - val_loss: 0.2762
Epoch 2/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 474ms/step - accuracy: 0.9894 - loss: 0.0277 - val_accuracy: 0.8462 - val_loss: 2.4498
Epoch 3/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 41s 480ms/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 1.0000 - val_loss: 4.9531e-04
Epoch 4/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 40s 469ms/step - accuracy: 0.9984 - loss: 0.0097 - val_accuracy: 0.6194 - val_loss: 126.6598
Epoch 5/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 41s 473ms/step - accuracy: 0.9894 - loss: 0.0400 - val_accuracy: 0.8947 - val_loss: 0.9456
Epoch 6/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 467ms/step - accuracy: 0.9808 - loss: 0.0675 - val_accuracy: 0.9919 - val_loss: 0.1531
Epoch 7/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 33s 467ms/step - accuracy: 0.9957 - loss: 0.0345 - val_accuracy: 0.9717 - val_loss: 0.0709
Epoch 8/75
70/70 ━━━━━━━━━━━━━━━━━━━━ 4

Test Accuracy: 100.00%


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
INPUT_SHAPE = IMAGE_SIZE + (3,)
DATASET_PATH = '/content/drive/MyDrive/forest fire'  # Folder containing 'fire/' and 'non_fire/'

# Data Generators with Splits
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=True
)



#X-MobileNet Model
def build_x_mobilenet(input_shape=(224, 224, 3), num_classes=1):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_x_mobilenet(INPUT_SHAPE)

# Training
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True),
    ModelCheckpoint('x_mobilenet_best.h5', save_best_only=True)
]

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks
)

# Save
model.save("x_mobilenet_final.h5")


Found 2469 images belonging to 2 classes.
Found 616 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 532ms/step - accuracy: 0.7623 - loss: 0.4906

78/78 ━━━━━━━━━━━━━━━━━━━━ 71s 763ms/step - accuracy: 0.7637 - loss: 0.4881 - val_accuracy: 0.9627 - val_loss: 0.1782
Epoch 2/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step - accuracy: 0.9681 - loss: 0.1030

78/78 ━━━━━━━━━━━━━━━━━━━━ 64s 601ms/step - accuracy: 0.9682 - loss: 0.1028 - val_accuracy: 0.9756 - val_loss: 0.1016
Epoch 3/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step - accuracy: 0.9857 - loss: 0.0540

78/78 ━━━━━━━━━━━━━━━━━━━━ 84s 627ms/step - accuracy: 0.9857 - loss: 0.0539 - val_accuracy: 0.9724 - val_loss: 0.0785
Epoch 4/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 81s 615ms/step - accuracy: 0.9900 - loss: 0.0344 - val_accuracy: 0.9724 - val_loss: 0.0858
Epoch 5/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step - accuracy: 0.9907 - loss: 0.0324

78/78 ━━━━━━━━━━━━━━━━━━━━ 49s 631ms/step - accuracy: 0.9907 - loss: 0.0323 - val_accuracy: 0.9756 - val_loss: 0.0747
Epoch 6/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 46s 597ms/step - accuracy: 0.9948 - loss: 0.0182 - val_accuracy: 0.9756 - val_loss: 0.0825
Epoch 7/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 47s 604ms/step - accuracy: 0.9917 - loss: 0.0222 - val_accuracy: 0.9756 - val_loss: 0.0841
Epoch 8/20
78/78 ━━━━━━━━━━━━━━━━━━━━ 48s 615ms/step - accuracy: 0.9982 - loss: 0.0143 - val_accuracy: 0.9692 - val_loss: 0.0964


In [ ]:
from PIL import Image
import numpy as np

# Load the image
img = Image.open('/content/drive/MyDrive/testing/download (1).jpg').convert('RGB')

img = img.resize((224, 224))
img.show()
img_array = np.array(img) / 255.0  # Normalize if needed
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

In [ ]:
# Make a prediction
prediction = model.predict(img_array)


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step


In [ ]:
class_labels = ['Fire', 'No fire']
predicted_class = class_labels[np.argmax(prediction)]
print(f"Predicted Class: {predicted_class}")



Predicted Class: Fire
